# CMDfit
Author: Seth Gossage  
Version: 0.1
***
Below are descriptions of the data, model, and statistical analysis (likelihood function, priors, posterior calculation) relevant to this code.

## Data
In my project, the data come from observations of invidual stars (e.g. $N$ such observations). We often delineate where a star is in its evolution by examining how bright and hot it is at the time of observation. Each data point is a measurement of the flux from a star, which is recorded as an apparent magnitude (apparent because this is how bright the star appears to us on Earth, and not how intrinsically bright it is). Thus,

* My data points are apparent magnitudes, which I will label as $m_{ij}$. The subscripting demarcates the magnitude taken in the $j^{th}$ wavelength filter (e.g. the magnitude in either the U, B, V, R, or I filter) and corresponding to the $i^{th}$ data point (so the magnitude recorded for one of the $N$ observations).  

* Each data point has recorded uncertainties based on the calibration of the instruments used to observe a given star within a given filter. I will call these $\sigma_{ij}$, and the subscripting works the same as above for the magnitudes.
***

## Model
My data are magnitude measurements of individual stars; to model these observations, I will then evolve models of individual stars and find their magnitudes for a range of ages, initial masses, chemical compositions (metallicities), and rotation velocity values. These are the primary parameters that I will work with in this project (more could be considered, but could be included in future projects, and I am mostly interested in the parameters of age and rotation velocity). Collections of these models at a fixed point in time are called isochrones, and they represent what a stellar cluster might look like if it where observed at that time. So isochrones are sets of model stars, with various initial masses, at a particular age.

* My model points are the magnitudes calculated from these stellar models. I will use $\mu_{ij}$ to symbolize them; the subscripting serves the same purpose as for the data points: $j$ to mark which filter I am considering and $i$ for the $i^{th}$ star's magnitude in that filter.
  
Given a set of these isochrones (each at a different age, metallicity, and intial rotation rate), we have a range of ages and other parameters to test with and find which isochrone best simulates the data.
***

## The Likelihood Function $p(D|\theta,M,I)$  

The likelihood function is the probability of a model successfully simulating a dataset. I am going to follow a similar framework to what is used in the paper by van Dyk et al., 2009 (http://arxiv.org/pdf/0905.2547.pdf).  

There are several complicating factors that will need to be considered, but as a preliminary setup, I can form a likelihood based on the following points.

* At a given time in a star's life it is expected to have a certain magnitude. (At later stages this is not true generally, star's pulsate dramatically and can go through periods of being relatively dim, bright, and then dim again. We are fitting isochrones primarily to the star's in earlier stages of their life, so this is not a major issue in this project.)  

* The uncertainties $\sigma_{ij}$ can be assumed Gaussian to be as non-comittal as possible, rather than making uneccesary, possibly inaccurate assumptions about the nature of these uncertainties.

* In this project we examine star's in a stellar cluster (the Hyades specifically) and so all stars are assumed to be at roughly the same distance, age, and share common initial rotation rates and metallicities. (This is based on the idea that stars in clusters form from a single cloud of dust, at roughly similar ages (within a number of years), and are also relatively close to each other as a consequence of this.)

* Additionally, extinction (magnitude shifts due to intervening dust between us and the cluster, which scatters light) is assumed to affect all stars in the cluster in the same way, so shifts due to it are irrelevant, as long as stars are in the same cluster. (This is mostly important to factor in when calculating magnitudes from the stellar evolution models.)

Therefore, matching measured magnitudes (data) to predicted magnitudes (model), a preliminary likelihood function may be written as:  

$$p(D | \theta, M, I) = \Pi^N_{i=1}\left(\Pi^n_{j=1}\left[\frac{1}{\sqrt{2\pi\sigma_{ij}^2}}\exp\left\{{\frac{-(m_{ij} - \mu_{ij})}{2\sigma_{ij}}}\right\}^2\right]\right)$$  

Combining individual likelihoods which are checking the probabilities between the $i^{th}$ predicted magnitude and its corresponding measured magnitude in each individual $j^{th}$ filter (so how far the measurement deviates from the prediction, considering uncertainties). So, I am also saying above that there could be as many as $n$ filters being considered in the likelihood.

I have calculated magnitudes available from stellar models, and recorded magnitudes, along with their uncertainties available from data tables. So this likelihood is calcuable.

### Complications  

The comparison between a given data and model point above is not as straight forward as I have described so far. For several reasons, the magnitude given by a data point could inherently violate the model. Here are the known reasons:

* The observed single star may actaully be two stars, i.e. a binary system. In this case comparing the magnitude of a binary system to the magnitude of a single star as assumed in the model calculations would be an inappropriate comparison.

* The observed star could actually be a "field star", rather than a member of the stellar cluster. Field stars are stars that just happen to have magnitudes that make them look like plausible candidates of the cluster, when in reality they are much brighter stars located much further away, or fainter stars located closer to us.  

Dealing with these complications can be done in the following ways: 

** Binary Systems  
(discussed in http://iopscience.iop.org/article/10.1088/0004-637X/696/1/12/pdf (DeGennaro et al, 2009), Sect. 2.2)
**
***
We can treat each computed magnitude from an isochrone as if it were a binary system. This does not change the likelihood written above, but it will change how we sample the prior involving the initial mass parameter for star (or now it should be called "system") $i$.  

** Field Stars  
(discussed in http://iopscience.iop.org/article/10.1088/0004-637X/696/1/12/pdf, Sect. 2.3
and http://arxiv.org/pdf/0905.2547.pdf (van Dyk, 2009), Sect. 3.3)  
**
***
This can be handled by adapting the likelihood function to a mixture model (i.e. a likelihood which considers the data as a mixture of cluster *and* field stars). So I will have a $P_{field}$, which is the probability that a data point is a field star (or the amplitude of the field star distrubution in this mixture). This modifies the likelihood a bit in the following way:  

$$p(D | \theta, M, I) = \Pi^N_{i=1}\left(\Pi^n_{j=1}\left[\frac{\left(1-P_{field}\right)}{\sqrt{2\pi\sigma_{ij}^2}}\exp\left\{{\frac{-(m_{ij} - \mu_{ij})}{2\sigma_{ij}}}\right\}^2\right]\right) + P_{field}\mathcal{L}_{field}$$  

(this is similar to the van Dyk paper's equation 5, but they define the probability concerning cluster membership in the reverse way)  where $\mathcal{L}_{field}$ is the contribution to the likelihood function coming from field stars. The van Dyk paper speaks about possible ways to treat this, and they use a uniform distribution defined as  

$$\mathcal{L}_{field} = \frac{1}{\Pi_{j=1}^n \left(max_j - min_j\right)}$$

*******

Here, $max_j$ is the maximum magnitude value in filter $j$, and $min_j$ is the minimum magnitude in this filter. I will adopt this for the time being, but it looks like improvements could be made here possibly.

With this form of the likelihood and a proper combination of priors, I can use MCMC to examine samples of the posterior distribution of the various parameters in my problem. I will be looking  particularly at the rotation velocity $\Omega$ and age $\tau$, in order to exmaine the relationship between these parameters.  

I have access to model magnitudes, data magnitudes and uncertainties, all in a range of filters, over a range of intitial masses, ages, metallicities, and rotation velocities. So this likelihood is calcuable as well.  

I will need to put it into code and run MCMC sampling with it and appropriate priors on the parameters listed (also including $P_{field}$ as a parameter) in order to determine the best fitting isochrone, and map out the posterior for the age and rotation rate of the Hyades data that I have available.


In [ ]:
# read in data points, read in model points, calculate likelihood.